In [ ]:
# Please put import here!
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import mutual_info_score
from sklearn.feature_selection import mutual_info_classif
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier  # Change to appropriate model
import numpy as np
from sklearn.linear_model import LinearRegression, LassoCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import seaborn as sns

## Data Preprocessing

In [ ]:
# Mount Google Drive (if needed)
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Load the flight and weather data
# Before run this code, please download the csv files and upload the files
df1 = pd.read_csv('/content/drive/MyDrive/new.csv')
df2 = pd.read_csv('/content/drive/MyDrive/WeatherEvents_Jan2016-Dec2022.csv')

In [ ]:
flight_data = df1.copy()
weather_data = df2.copy()
# Ensure both 'Flight_Date' columns are in datetime format (without time part)
flight_data['Flight_Date'] = pd.to_datetime(flight_data['Flight_Date']).dt.date
weather_data['Flight_Date'] = pd.to_datetime(weather_data['StartTime(UTC)']).dt.date

# Clean city and state data
flight_data['Origin_State'] = flight_data['Origin_City'].apply(lambda x: x.split(',')[1].strip())
flight_data['Origin_City'] = flight_data['Origin_City'].apply(lambda x: x.split(',')[0].strip())

weather_data['Origin_City'] = weather_data['City'].str.strip()
weather_data['Origin_State'] = weather_data['State'].str.strip()

# Perform a left join to keep all flight data, even if no matching weather event
combined_data = pd.merge(
    flight_data,
    weather_data[['Flight_Date', 'Origin_City', 'Origin_State', 'Type', 'Severity', 'Precipitation(in)']],
    on=['Flight_Date', 'Origin_City', 'Origin_State'],
    how='left'
)

## Final Model

1. Data Cleaning
* Categorical -> numerical

In [ ]:
from sklearn.preprocessing import LabelEncoder

# Create a LabelEncoder object
le = LabelEncoder()

# Specify the columns to convert
categorical_cols = ['Airline_Name', 'Origin_City', 'Origin_State', 'Destination_City',
                    'Cancellation_Code', 'Cancelled_Flag', 'Diverted_Flag', 'Type', 'Severity']

# Replace NaN with 0 and apply label encoding to each column
for col in categorical_cols:
    combined_data[col] = combined_data[col].fillna(0)  # Replace NaN with 0
    combined_data[col] = le.fit_transform(combined_data[col].astype(str))  # Encode column


* One hot key encoding - date

In [ ]:
combined_data['Year'] = pd.to_datetime(combined_data['Flight_Date']).dt.year
combined_data['Month'] = pd.to_datetime(combined_data['Flight_Date']).dt.month
combined_data['DayOfMonth'] = pd.to_datetime(combined_data['Flight_Date']).dt.day
combined_data['DayOfWeek'] = pd.to_datetime(combined_data['Flight_Date']).dt.day_name()
combined_data = combined_data[combined_data['Year'] < 2023]
combined_data.drop('Flight_Date', axis=1, inplace=True)

<ipython-input-6-31fef0d917ee>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  combined_data.drop('Flight_Date', axis=1, inplace=True)


In [ ]:
combined_data = pd.get_dummies(combined_data, columns=['DayOfWeek', 'Month'], dtype=int)

#replace the column name of the DayOfWeek columns into the weekday names
weather = {''}
mapper_week = {'DayOfWeek_Monday': 'Monday', 'DayOfWeek_Tuesday': 'Tuesday', 'DayOfWeek_Wednesday': 'Wednesday',
                   'DayOfWeek_Thursday': 'Thursday', 'DayOfWeek_Friday': 'Friday', 'DayOfWeek_Saturday': 'Saturday',
                   'DayOfWeek_Sunday': 'Sunday'}
combined_data.rename(columns=mapper_week,inplace=True)
# replace the column name of the Month columns into the month names
mapper_month = {'Month_1': 'January', 'Month_2': 'February', 'Month_3': 'March',
                    'Month_4': 'April', 'Month_5': 'May', 'Month_6': 'June',
                    'Month_7': 'July', 'Month_8': 'August', 'Month_9': 'September',
                    'Month_10': 'October', 'Month_11': 'November', 'Month_12': 'December'}
combined_data.rename(columns=mapper_month, inplace=True)

#drop one of the columns to avoid multicollinearity
combined_data.drop(['Monday', 'January'], axis=1, inplace=True)

* HH:MM into minutes

In [ ]:
def convert_to_min(time_str):
    if pd.isna(time_str):  # Check for NaN values
        return 0.0
    hours, minutes = map(int, time_str.split(':'))
    return hours * 60 + minutes

# Apply the function to each delay column
combined_data['Carrier_Delay_Hours'] = combined_data['Carrier_Delay_HH_MM'].apply(convert_to_min)
combined_data['Weather_Delay_Hours'] = combined_data['Weather_Delay_HH_MM'].apply(convert_to_min)
combined_data['NAS_Delay_Hours'] = combined_data['NAS_Delay_HH_MM'].apply(convert_to_min)
combined_data['Security_Delay_Hours'] = combined_data['Security_Delay_HH_MM'].apply(convert_to_min)
combined_data['Late_Aircraft_Delay_Hours'] = combined_data['Late_Aircraft_Delay_HH_MM'].apply(convert_to_min)

combined_data.drop(['Carrier_Delay_HH_MM', 'Weather_Delay_HH_MM', 'NAS_Delay_HH_MM',
                    'Security_Delay_HH_MM', 'Late_Aircraft_Delay_HH_MM'], axis=1, inplace=True)


In [ ]:
combined_data.drop(['Scheduled_Departure_Time','Actual_Departure_Time','Departure_Delay_Minutes','Scheduled_Arrival_Time','Actual_Arrival_Time','Arrival_Delay_Minutes'], axis=1, inplace=True)
combined_data.head()

,Airline_Name,Origin_City,Destination_City,Cancelled_Flag,Cancellation_Code,Diverted_Flag,Scheduled_Elapsed_Time_Minutes,Actual_Elapsed_Time_Minutes,Origin_State,Type,...,August,September,October,November,December,Carrier_Delay_Hours,Weather_Delay_Hours,NAS_Delay_Hours,Security_Delay_Hours,Late_Aircraft_Delay_Hours
0,10,109,41,1,2,1,184.0,255.0,8,0,...,0,0,0,0,1,1564,284,64,98,217
1,2,180,213,1,2,1,282.0,283.0,32,0,...,0,0,0,0,0,1838,868,170,279,503
3,4,229,279,1,2,1,87.0,78.0,33,0,...,0,0,1,0,0,2692,288,279,169,638
6,2,336,81,1,2,1,199.0,200.0,7,0,...,1,0,0,0,0,1074,86,642,236,164
8,14,270,295,1,2,1,84.0,89.0,32,2,...,0,0,0,0,1,1642,769,555,126,136


* delay time

In [ ]:
delay_threshold_min = 180    # 3 hours is based on Recommended Check-in time (prior to boarding time) from airlines
temp1 = combined_data[combined_data["Carrier_Delay_Hours"] < delay_threshold_min]["Carrier_Delay_Hours"]
temp2 = combined_data[combined_data["Carrier_Delay_Hours"] >= delay_threshold_min]["Carrier_Delay_Hours"]
print(temp1.count())
print(temp2.count())

18870
99760


In [ ]:
x = combined_data["Carrier_Delay_Hours"]
print(x[x>=delay_threshold_min])
print(x[x<delay_threshold_min])

x = combined_data["Carrier_Delay_Hours"] - delay_threshold_min   # Make values negative based on threshold
#combined_data["Carrier_Delay_Hours"]

y = (x - x.abs())/(2*x.abs()) + 1
print(y[y==1])
print(y[y==0])

0         1564
1         1838
3         2692
6         1074
8         1642
          ... 
126298    2008
126299    2008
126300    2008
126301    2779
126302    1611
Name: Carrier_Delay_Hours, Length: 99760, dtype: int64
17         18
18          2
20        130
21        130
29         78
         ... 
126217    115
126218    115
126219    115
126263      5
126282     16
Name: Carrier_Delay_Hours, Length: 18870, dtype: int64
0         1.0
1         1.0
3         1.0
6         1.0
8         1.0
         ... 
126298    1.0
126299    1.0
126300    1.0
126301    1.0
126302    1.0
Name: Carrier_Delay_Hours, Length: 99664, dtype: float64
17        0.0
18        0.0
20        0.0
21        0.0
29        0.0
         ... 
126217    0.0
126218    0.0
126219    0.0
126263    0.0
126282    0.0
Name: Carrier_Delay_Hours, Length: 18870, dtype: float64


In [ ]:
delay_list = ["Carrier_Delay_Hours", "Weather_Delay_Hours", "NAS_Delay_Hours", "Security_Delay_Hours", "Late_Aircraft_Delay_Hours"]

for delay_factor in delay_list:
  label = delay_factor.split("_")[0] + "_delayed"
  if(label == "Late_delayed"): label = "Late_Aircraft_delayed"
  combined_data[label] = (combined_data[delay_factor] - combined_data[delay_factor].abs())/(2*combined_data[delay_factor].abs()) + 1


In [ ]:
delay_list = ["Carrier_Delay_Hours", "Weather_Delay_Hours", "NAS_Delay_Hours", "Security_Delay_Hours", "Late_Aircraft_Delay_Hours"]

for delay_factor in delay_list:
    # Create a label column name
    label = delay_factor.split("_")[0] + "_delayed"
    if label == "Late_delayed":  # Adjust for "Late_Aircraft" naming
        label = "Late_Aircraft_delayed"

    # Create binary labels: 1 if delay > 180, else 0
    combined_data[label] = (combined_data[delay_factor] > 180).astype(int)



In [ ]:
combined_data.drop(delay_list, axis=1, inplace=True)
combined_data[["Carrier_delayed", "Weather_delayed", "NAS_delayed", "Security_delayed", "Late_Aircraft_delayed"]]

,Carrier_delayed,Weather_delayed,NAS_delayed,Security_delayed,Late_Aircraft_delayed
0,1,1,0,0,1
1,1,1,0,1,1
3,1,1,1,0,1
6,1,0,1,1,0
8,1,1,1,0,0
...,...,...,...,...,...
126298,1,1,1,1,1
126299,1,1,1,1,1
126300,1,1,1,1,1
126301,1,0,1,0,0


In [ ]:
combined_data['delayTime'] = combined_data['Actual_Elapsed_Time_Minutes'] - combined_data['Scheduled_Elapsed_Time_Minutes']
combined_data.drop(['Actual_Elapsed_Time_Minutes','Scheduled_Elapsed_Time_Minutes'], axis=1, inplace=True)

In [ ]:
combined_data.Carrier_delayed.unique()

array([1, 0])

3. Training Model

Create train and test data

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
features = ['Airline_Name', 'Origin_City', 'Destination_City', 'Cancelled_Flag',
            'Cancellation_Code', 'Diverted_Flag', 'Origin_State', 'Type',
            'Severity', 'Precipitation(in)', 'Year', 'DayOfMonth', 'Friday',
            'Saturday', 'Sunday', 'Thursday', 'Tuesday', 'Wednesday', 'February',
            'March', 'April', 'May', 'June', 'July', 'August', 'September',
            'October', 'November', 'December', 'Carrier_delayed', 'Weather_delayed',
            'NAS_delayed', 'Security_delayed', 'Late_Aircraft_delayed']
X = combined_data[features]
y = combined_data['delayTime']

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Linear Regression with Lasso regularization

In [ ]:
lasso = LassoCV()
lasso.fit(X_train, y_train)
y_lasso = lasso.predict(X_test)

rmse_lasso = np.sqrt(mean_squared_error(y_test, y_lasso))

print(f"Root Mean Squared Error of linear regression with Lasso regularization: {rmse_lasso}")

Root Mean Squared Error of linear regression with Lasso regularization: 20.1756709253371


RF

In [ ]:
# Define and train the Random Forest Regressor
rf_regressor = RandomForestRegressor(
        random_state = 2,
        max_depth = 10,
        n_estimators = 150,
        max_samples = 0.8,
        bootstrap = True
    )
rf_regressor.fit(X_train, y_train)

# Predict
y_rf = rf_regressor.predict(X_test)

rmse_rf = np.sqrt(mean_squared_error(y_test, y_rf))

# Print evaluation metrics
print("Root Mean Squared Error of random forest with hypertuning:", rmse_rf)


Root Mean Squared Error of random forest with hypertuning: 14.60543554535095


CART

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import mean_absolute_error

cart_model = DecisionTreeClassifier(
    random_state=1,
    max_depth=50,
    min_samples_split=2,
    min_samples_leaf=1,
    max_features = None,
    criterion='entropy'
)
cart_model.fit(X_train, y_train)
y_cart = cart_model.predict(X_test)
rmse_cart = np.sqrt(mean_squared_error(y_test, y_cart))
# Print evaluation metrics
print("Root Mean Squared Error of CART:", rmse_cart)
mae = mean_absolute_error(y_test, y_cart)
print(mae)

Root Mean Squared Error of CART: 11.692172758670461
3.874652280198938


In [ ]:
from sklearn.model_selection import GridSearchCV

# Define hyperparameter grid
param_grid = {
    'max_depth': [10, 20, 50, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 5, 10],
    'max_features': ['sqrt', 'log2', None],
    'criterion': ['gini', 'entropy']
}

# Initialize grid search
grid_search = GridSearchCV(
    estimator=DecisionTreeClassifier(random_state=1),
    param_grid=param_grid,
    cv=5,  # 5-fold cross-validation
    scoring='accuracy',  # Adjust based on your metric
    verbose=1,
    n_jobs=-1
)

# Fit grid search
grid_search.fit(X_train, y_train)
print("Best parameters:", grid_search.best_params_)


Fitting 5 folds for each of 216 candidates, totalling 1080 fits


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:776: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


Best parameters: {'criterion': 'entropy', 'max_depth': 50, 'max_features': None, 'min_samples_leaf': 1, 'min_samples_split': 2}


Gradient Boosting

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
gb_model = GradientBoostingClassifier(n_estimators=20, random_state=0)
gb_model.fit(X_train, y_train)
y_gb = gb_model.predict(X_test)
rmse_gb = np.sqrt(mean_squared_error(y_test, y_gb))
# Print evaluation metrics
print("Root Mean Squared Error of Gradient Boosting:", rmse_gb)

KeyboardInterrupt: 

What-if Scenarios

Delay Factor

In [ ]:
import pandas as pd

def create_test_data(df, is_delay, features, delay_type):
    # Create a copy of the DataFrame to avoid modifying the original
    df_copy = df.copy()
    df_copy[delay_type] = is_delay
    x = df_copy[features]
    y = df_copy[delay_type]
    return x, y

def simulate_factor(df, features, factor, model):
    result = []

    # Filter rows based on the factor's condition
    delay_df = df.loc[df[factor] == 1]
    non_delay_df = df.loc[df[factor] == 0]

    # Create test data for delayed and non-delayed cases
    test_delay, y_not_delay = create_test_data(non_delay_df, 1, features, factor)
    test_not_delay, y_delay = create_test_data(delay_df, 0, features, factor)

    # Predict using the model
    simulate_delay = model.predict(test_delay)
    simulate_not_delay = model.predict(test_not_delay)

    # Calculate differences between actual and simulated means
    result.append(y_not_delay.mean() - simulate_delay.mean())
    result.append(y_delay.mean() - simulate_not_delay.mean())

    return result

# Simulating factors
carrier_result = simulate_factor(pd.concat([X_test, y_test], axis=1), features, "Carrier_delayed", cart_model)
print(f"Carrier delay exists: {carrier_result[0]:.2f}, Carrier delay does not exist: {carrier_result[1]:.2f}")

security_result = simulate_factor(pd.concat([X_test, y_test], axis=1), features, "Security_delayed", cart_model)
print(f"Security delay exists: {security_result[0]:.2f}, Security delay does not exist: {security_result[1]:.2f}")

NAS_result = simulate_factor(pd.concat([X_test, y_test], axis=1), features, "NAS_delayed", cart_model)
print(f"NAS delay exists: {NAS_result[0]:.2f}, NAS delay does not exist: {NAS_result[1]:.2f}")

weather_result = simulate_factor(pd.concat([X_test, y_test], axis=1), features, "Weather_delayed", cart_model)
print(f"Weather delay exists: {weather_result[0]:.2f}, weather delay does not exist: {weather_result[1]:.2f}")

late_aircraft_result = simulate_factor(pd.concat([X_test, y_test], axis=1), features, "Late_Aircraft_delayed", cart_model)
print(f"Late aircraft delay exists: {late_aircraft_result[0]:.2f}, late aircraft delay does not exist: {late_aircraft_result[1]:.2f}")


Carrier delay exists: 5.63, Carrier delay does not exist: 6.02
Security delay exists: 7.49, Security delay does not exist: 5.70
NAS delay exists: 9.09, NAS delay does not exist: 2.92
Weather delay exists: 6.03, weather delay does not exist: 5.75
Late aircraft delay exists: 6.15, late aircraft delay does not exist: 5.80


Weather Type

In [ ]:
def create_test_data(df, is_delay, features, delay_type):
    # Create a copy of the DataFrame to avoid modifying the original
    df_copy = df.copy()
    df_copy[delay_type] = is_delay
    x = df_copy[features]
    y = df_copy[delay_type]
    return x, y

def simulate_factor(df, features, value, factor, model):
    result = []

    # Filter rows based on the factor's condition
    delay_df = df.loc[df[factor] == value]
    non_delay_df = df.loc[df[factor] == 0]

    # Create test data for delayed and non-delayed cases
    test_delay, y_not_delay = create_test_data(non_delay_df, value, features, factor)
    test_not_delay, y_delay = create_test_data(delay_df, 0, features, factor)

    # Predict using the model
    simulate_delay = model.predict(test_delay)
    simulate_not_delay = model.predict(test_not_delay)

    # Calculate differences between actual and simulated means
    result.append(y_not_delay.mean() - simulate_delay.mean())
    result.append(y_delay.mean() - simulate_not_delay.mean())

    return result
# Simulating factors
cold_result = simulate_factor(pd.concat([X_test, y_test], axis=1), features, 1, "Type", cart_model)
print(f"Cold weather event exists: {cold_result[0]:.2f}, Cold weather event does not exist: {cold_result[1]:.2f}")

fog_result = simulate_factor(pd.concat([X_test, y_test], axis=1), features, 2, "Type", cart_model)
print(f"Fog weather event exists: {fog_result[0]:.2f}, Fog weather event does not exist: {fog_result[1]:.2f}")

hail_result = simulate_factor(pd.concat([X_test, y_test], axis=1), features, 3, "Type", cart_model)
print(f"Hail weather event exists: {hail_result[0]:.2f}, Hail weather event does not exist: {hail_result[1]:.2f}")

precipitation_result = simulate_factor(pd.concat([X_test, y_test], axis=1), features, 4, "Type", cart_model)
print(f"Precipitation weather event exists: {precipitation_result[0]:.2f}, Precipitation weather event does not exist: {precipitation_result[1]:.2f}")

rain_result = simulate_factor(pd.concat([X_test, y_test], axis=1), features, 5, "Type", cart_model)
print(f"Rain weather event exists: {rain_result[0]:.2f}, Rain weather event does not exist: {rain_result[1]:.2f}")

snow_result = simulate_factor(pd.concat([X_test, y_test], axis=1), features, 6, "Type", cart_model)
print(f"Snow weather event exists: {snow_result[0]:.2f}, Snow weather event does not exist: {snow_result[1]:.2f}")

storm_result = simulate_factor(pd.concat([X_test, y_test], axis=1), features, 7, "Type", cart_model)
print(f"Storm weather event exists: {storm_result[0]:.2f}, Storm weather event does not exist: {storm_result[1]:.2f}")

Cold weather event exists: 7.46, Cold weather event does not exist: 9.26
Fog weather event exists: 8.39, Fog weather event does not exist: 6.44
Hail weather event exists: 9.36, Hail weather event does not exist: 17.91
Precipitation weather event exists: 10.29, Precipitation weather event does not exist: 4.27
Rain weather event exists: 11.29, Rain weather event does not exist: 6.01
Snow weather event exists: 12.19, Snow weather event does not exist: 0.20
Storm weather event exists: 13.20, Storm weather event does not exist: 12.26
